<a href="https://colab.research.google.com/github/smenon02/mutations/blob/main/binary_mutations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
from datetime import date

today = date.today()
print("Last updated on", today)

Last updated on 2024-10-22


In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
root_dir = "/content/drive/My Drive/"
project_folder = "Mutations Research/"
# change working directory
os.chdir(root_dir + project_folder)
# confirm working directory
%pwd

Mounted at /content/drive


'/content/drive/My Drive/Mutations Research'

In [ ]:
# import libraries
import pandas as pd
import time
import sklearn
import scipy
import sys
import numpy as np
import dill
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

ModuleNotFoundError: No module named 'dill'

# Data Processing

In [ ]:
def remove_lowVar(df, minVariance):
  selector = VarianceThreshold(threshold = minVariance)
  selector.fit(df)
  df_new = df.loc[:, selector.get_support()]
  print("Removed {} of {} for having zero variance".format(df.shape[1]- df_new.shape[1], df.shape[1]),flush=True)
  return(df_new)

In [ ]:
# Read in data
ccle_mutations = remove_lowVar(pd.read_csv('./Datasets/CCLE_mutations_bool_damaging.csv', index_col=0),0)
ccle_met = remove_lowVar(pd.read_csv('./Datasets/CCLE_metabolomics_averages.csv',index_col=0),0)

Removed 146 of 17349 for having zero variance
Removed 0 of 225 for having zero variance


In [ ]:
#Map mutation broad IDs to CCLE cell line names
ccle_map_info = pd.read_csv('./Datasets/sample_info.csv')
ccle_map = ccle_map_info.set_index('DepMap_ID')['cell_line_name'].to_dict()
ccle_mutations.index = ccle_mutations.index.map(ccle_map)

In [ ]:
#Merge data
ccle_merge = pd.merge(ccle_mutations,ccle_met, left_index=True, right_index=True)

In [ ]:
#Store mutation genes and metabolites
mutation_genes = ccle_mutations.columns
metabolites = ccle_met.columns

# Create models

## RF Model

In [ ]:
scaler = StandardScaler()
def train_multiRegressCV(classifier, data, x_cols, y_cols, n_splits=5, pval_cutoff=0.05, scale_y=True):

  # split into train and test data: all metabolites, median data
  X =  data.loc[:, x_cols]
  y =  data.loc[:, y_cols]

  # create multi-output ridge regression model
  if classifier=="RF":
    model=RandomForestRegressor(random_state=0, n_estimators=100, n_jobs=-1, max_depth=10)
    print("Training model with random forest!")

  else:
    print("Enter model type!")
    sys.exit()

  clf = MultiOutputRegressor(model)

  cv = KFold(n_splits=n_splits,
                shuffle=True,
                random_state=123)

  # create empty lists to store CV scores, confusion mat, etc.
  df_y_test_all = pd.DataFrame(columns=y_cols)
  df_y_pred_all = pd.DataFrame(columns=y_cols)

  count = 1

  # loop through cross-val folds
  all_means = pd.DataFrame(index=x_cols)

  for train_index, test_index in cv.split(X, y):
      start_time = time.perf_counter()
      print(count,flush=True)
      X_trainCV, X_testCV = X.iloc[train_index], X.iloc[test_index]
      y_trainCV, y_testCV = y.iloc[train_index], y.iloc[test_index]

      # X_trainCV =  scaler.fit_transform(X_trainCV)
      # X_testCV =  scaler.transform(X_testCV)

      if scale_y:
        y_trainCV =  scaler.fit_transform(y_trainCV)
        y_testCV =  scaler.transform(y_testCV)

      tmp_mdl = clf.fit(X_trainCV, y_trainCV)
      y_predCV = tmp_mdl.predict(X_testCV)

      df_pred_temp  = pd.DataFrame(y_predCV, columns=y_cols)
      df_test_temp = pd.DataFrame(y_testCV, columns=y_cols)

      df_y_test_all = pd.concat([df_y_test_all, df_test_temp])
      df_y_pred_all = pd.concat([df_y_pred_all, df_pred_temp])

      count = count+1

      finish_time = time.perf_counter()
      print("CV fold finished in {} seconds".format(finish_time-start_time),flush=True)

  #print("Shape of y_test:")
  #print(df_y_test_all.shape)
  #print("Calculating Pearson's R for each metabolite...")
  r = []
  for i, col in enumerate(y_cols):
    r.append(scipy.stats.pearsonr(df_y_test_all.iloc[:,i], df_y_pred_all.iloc[:,i]))

  df_results = pd.DataFrame(r, columns=['R','pval'], index=y_cols)  # index=y_cols,

  #print("Shape of results:")
  #print(df_results.shape)


  df_results['Significant']  = (df_results.pval < pval_cutoff) & (df_results.R > 0)
  df_results['R2']  = df_results.R**2

  # X_final = scaler.fit_transform(X)
  X_final = pd.DataFrame(X_final,columns=X.columns,index=X.index)
  if scale_y:
    y_final = scaler.fit_transform(y)
    y_final = pd.DataFrame(y_final,columns=y.columns,index=y.index)

  print("Training final model on dataset of {} samples and {} features".format(X_final.shape[0], X_final.shape[1]))

  final_mdl = clf.fit(X_final, y_final)

  return df_results, final_mdl

# Build Models

In [ ]:
#Build model
modeltype = 'RF'
data_in = ccle_merge
x_cols = mutation_genes
y_cols = metabolites
RF_results, RF_mdl = train_multiRegressCV(modeltype, data_in, x_cols, y_cols, n_splits=5, pval_cutoff=0.05, scale_y=False)

In [ ]:
#Export results
RF_results.to_csv('./Results/RF_BinaryMutations_Correlations.csv', index=False)
with open('./Results/RF_BinaryMutations_Model.pkl', 'wb') as file:
  pickle.dump(RF_mdl, file)